<a href="https://colab.research.google.com/github/melilucmel/proyecto-google-ads/blob/main/ProyectoParteII%2BLuceroAntonietti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
df_googleads_clean = pd.read_csv("https://raw.githubusercontent.com/melilucmel/proyecto-google-ads/refs/heads/main/GoogleAds_Clean.csv")
